In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import chromedriver_binary
from io import BytesIO
import base64
from python3_anticaptcha import ImageToTextTask, CallbackClient
from PIL import Image
from time import sleep
import requests
import time
import config as cg

idCard = cg.IDCardInfo()
anticaptcha = cg.AntiCaptchaInfo()

# 前臺開啟瀏覽器模式
def openChrome():
    # 加啟動配置
    option = webdriver.ChromeOptions()
    option.add_argument('disable-infobars')
    option.add_argument("--window-size=1280,1024")
    # 開啟chrome瀏覽器
    driver = webdriver.Chrome(chrome_options=option)
    return driver

# 輸入查詢表單
def insertIDCardReissue(driver):
    url = "https://www.ris.gov.tw/app/portal/3043"
    driver.get(url)
    driver.implicitly_wait(15)
    driver.switch_to.frame("content-frame")
    
    # 找到身分證字號輸入框並輸入查詢內容
    driver.find_element_by_id("idnum").send_keys(idCard.id)
    
    # 找到發證日期 下拉選單
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyTWY')).select_by_value(idCard.applyTWY)
    
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyMM')).select_by_value(idCard.applyMM)
    
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyDD')).select_by_value(idCard.applyDD)
        
    #發證地點：
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('siteId')).select_by_value(idCard.siteId)

    #領補換類別：
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyReason')).select_by_value(idCard.applyReason)

    #圖片驗證
    captchaInput_captcha = driver.find_element_by_id("captchaInput_captcha-refresh")
    captchaInput_captcha.send_keys("")
    
    #captcha = driver.find_element_by_id("captchaImage_captcha-refresh")
    path = 'captchaImage.png'
    get_captcha(driver, path)
    captcha_key = predict_captcha(path)
    captchaInput_captcha.send_keys(captcha_key)
    
    result = None
    while result is None:
        driver.find_element_by_xpath("/html/body/div/div[4]/div[1]/div/form/div[4]/button[1]").click()
        try:
            result = send_form(driver, path)
        except:
            pass

    print(result)
        
def send_form(driver, path):
    
    try:
        driver.find_element_by_id("resultBlock")
    
    except:
        
        driver.set_window_size(1280, 1024)
        captchaInput = driver.find_element_by_id("captchaInput_captcha-refresh")
        captchaInput.clear()
        get_captcha(driver, path)
        captcha_key = predict_captcha(path)
        print("send_form.predict_captcha.captcha_key = ", captcha_key)
        
        captchaInput.send_keys(captcha_key)
    
    else:
        return '表單送出成功'
        

def get_captcha(driver, path):
    scroll_width = driver.execute_script('return document.body.parentNode.scrollWidth')
    scroll_height = driver.execute_script('return document.body.parentNode.scrollHeight')
    driver.set_window_size(scroll_width, scroll_height) #the trick
    time.sleep(1)
    png = driver.get_screenshot_as_png()         # 先將目前的 screen 存起來
    image = Image.open(BytesIO(png))
    image = image.crop((350, 200, 580, 290)) # 根據位置剪裁圖片
    image.save(path)               # 重新儲存圖片為 png 格式

In [2]:
import cv2
from keras.models import Model, load_model
from skimage.util import invert
import numpy as np

def predict_captcha(image_path):
    
    #image to black and white
    originalImage = cv2.imread(image_path)
    grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
    (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 130, 255, cv2.THRESH_BINARY)
    image = np.stack(invert(np.array(blackAndWhiteImage))/255.0)
    image = image[np.newaxis, :, :]
    print('image.shape =', image.shape)
    filepath = 'cnn_model.hdf5'
    model = load_model(filepath)
    prediction = model.predict(image)
    captcha = prediction_to_text(prediction)
    return captcha

In [3]:
dic = {'2':0, '3':1, '4':2, '5':3,'6':4, '7':5, '8':6, '9':7, 
         'A': 8, 'B':9,'C':10, 'D':11, 'E':12, 'F':13, 'G':14, 'H':15,
         'J':16, 'K':17, 'N':18, 'P':19,'Q':20, 'R':21, 'S':22, 'T':23,
         'U':24, 'V':25, 'X':26, 'Y':27, 'Z':28}

new_dict = {v : k for k, v in dic.items()}

def prediction_to_text(prediction):
    res = ''
    for arr in prediction:
        arr = np.around(arr, decimals=1)
        arr = arr[0, :].tolist()
        #print('arr = ', arr)
        index = arr.index(max(arr))
        #print('index =', index)
        key = new_dict[index]
        #print('key = ', key)
        res = res+key
    return res

In [4]:
driver=openChrome()
insertIDCardReissue(driver)

<ipython-input-1-71e45dd0a9ab>:23: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=option)


image.shape = (1, 90, 230)
表單送出成功
